#Paquetes necesarios

In [ ]:
import cv2  
import math 

from ultralytics import YOLO



Desde cámara, detección con yolov8 y modelo nano. Visualización propia

In [ ]:
# Carga del modelo
model = YOLO('yolov8n.pt') #Contenedores

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde lawebcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Diversos modelos preentrenados, visualizando

In [ ]:
# Carga del modelo
#model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
results = model(filename, show=True)



Reconocimiento decaracteres tras instalar easyocr

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
#result = reader.readtext('toy.tif')
result = reader.readtext('matricula.jpeg')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')


DETECTOR DE MATRICULAS FOTO

In [ ]:
import cv2
import math
import easyocr
from ultralytics import YOLO

# Carga del modelo YOLO
model = YOLO('yolov8n.pt')

# Carga del modelo OCR para la detección de matrículas
reader = easyocr.Reader(['en'])

# Nombre de las distintas clases de YOLO
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# Ruta de la imagen que deseas cargar
image_path = 'coche3.png'
img = cv2.imread(image_path)

# Realiza inferencia en la imagen con YOLO
results = model(img)

# Para cada detección
for r in results:
    boxes = r.boxes

    for box in boxes:
        # Contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir a valores enteros

        # Clase
        cls = int(box.cls[0])
        class_name = classNames[cls]

        # Solo continuamos si la clase es un vehículo
        if class_name in ["car", "motorbike", "bus"]:
            # Recorta la región de interés (ROI) que contiene el vehículo
            vehicle_roi = img[y1:y2, x1:x2]

            # Detección de matrícula con OCR
            results_ocr = reader.readtext(vehicle_roi)

            # Si se encuentra texto (matrícula)
            if results_ocr:
                letras = results_ocr[0][1]
                numeros = results_ocr[1][1]
                print(f"Matrícula del {class_name}: {numeros} {letras}")

        # Dibuja el contenedor y clase
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Muestra la imagen con las detecciones
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


DETECTOR DE MATRICULAS VIDEO

In [1]:
import cv2
import time
import easyocr
from ultralytics import YOLO

# Carga del modelo YOLO
model = YOLO('yolov8n.pt')

# Carga del modelo OCR para la detección de matrículas
reader = easyocr.Reader(['en'])

# Nombre de las distintas clases de YOLO
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# Ruta del video que deseas cargar
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

# Establece un retraso de 100 milisegundos (0.1 segundos) entre fotogramas
delay = 100

while True:
    # Captura el fotograma del video
    ret, img = cap.read()

    if not ret:
        print("Fin del video.")
        break

    # Realiza inferencia en el fotograma con YOLO
    results = model(img)

    # Para cada detección
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir a valores enteros

            # Clase
            cls = int(box.cls[0])
            class_name = classNames[cls]

            # Solo continuamos si la clase es un vehículo
            if class_name in ["car", "motorbike", "bus"]:
                # Recorta la región de interés (ROI) que contiene el vehículo
                vehicle_roi = img[y1:y2, x1:x2]

                # Detección de matrícula con OCR
                results_ocr = reader.readtext(vehicle_roi)

                # Si se encuentra texto (matrícula)
                if results_ocr:
                    #letras = results_ocr[0][1]
                    #numeros = results_ocr[1][1]
                    print(f"Matrícula del {class_name}: {results_ocr}")

            # Dibuja el contenedor y clase
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Muestra el fotograma con las detecciones
    cv2.imshow('Video', img)

    # Introduce un retraso entre fotogramas
    time.sleep(delay / 1000.0)

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera los recursos
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 